<a href="https://colab.research.google.com/github/HamzaWajid1/BBC_news_classification_using_LLama2-7f-hb_finetuning/blob/main/Llama_2_7_b_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q accelerate==0.21.0 #— progress-bar off
!pip install -q peft==0.4.0  #— progress-bar off
!pip install -q bitsandbytes==0.40.2 #— progress-bar off
!pip install -q transformers==4.31.0 #— progress-bar off
!pip install -q trl==0.4.7 #— progress-bar off

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00


In [2]:
import os
from random import randrange
from functools import partial
import torch
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
AutoTokenizer,
BitsAndBytesConfig,
HfArgumentParser,
Trainer,
TrainingArguments,
DataCollatorForLanguageModeling,
EarlyStoppingCallback,
pipeline,
logging,
set_seed)

import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM
from trl import SFTTrainer
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
def create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype):
  bnb_config = BitsAndBytesConfig(
  load_in_4bit = load_in_4bit,
  bnb_4bit_use_double_quant = bnb_4bit_use_double_quant,
  bnb_4bit_quant_type = bnb_4bit_quant_type,
  bnb_4bit_compute_dtype = bnb_4bit_compute_dtype,
 )
  return bnb_config

In [5]:
def load_model(model_name, bnb_config):
#Loads model and model tokenizer

#:param model_name: Hugging Face model name
#:param bnb_config: Bitsandbytes configuration"""

# Get number of GPU device and set maximum memory
 n_gpus = torch.cuda.device_count()
 max_memory = f'{40960}MB'

 # Load model
 model = AutoModelForCausalLM.from_pretrained(
 model_name,
 quantization_config = bnb_config,
 device_map = 'auto', # dispatch the model efficiently on the available resources
 max_memory = {i: max_memory for i in range(n_gpus)},
 )

 # Load model tokenizer with the user authentication token
 tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token = True)

 # Set padding token as EOS token
 tokenizer.pad_token = tokenizer.eos_token
 return model, tokenizer

In [6]:
model_name='meta-llama/Llama-2-7b-hf'

In [7]:
load_in_4bit = True
bnb_4bit_use_double_quant = True
bnb_4bit_quant_type = 'nf4'
bnb_4bit_compute_dtype = torch.bfloat16

In [8]:
bnb_config = create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype)

In [9]:
model, tokenizer = load_model(model_name, bnb_config)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [10]:
import pandas as pd

In [11]:
df1=pd.read_csv('bbc-text.csv')

In [12]:
df1['Instruction']='Categorize the news article into one of the 5 categories:\n\ntech\nbusiness\nsport\nentertainment\npolitics'

In [13]:
df1.head(10)

,category,text,Instruction
0,tech,tv future in the hands of viewers with home th...,Categorize the news article into one of the 5 ...
1,business,worldcom boss left books alone former worldc...,Categorize the news article into one of the 5 ...
2,sport,tigers wary of farrell gamble leicester say ...,Categorize the news article into one of the 5 ...
3,sport,yeading face newcastle in fa cup premiership s...,Categorize the news article into one of the 5 ...
4,entertainment,ocean s twelve raids box office ocean s twelve...,Categorize the news article into one of the 5 ...
5,politics,howard hits back at mongrel jibe michael howar...,Categorize the news article into one of the 5 ...
6,politics,blair prepares to name poll date tony blair is...,Categorize the news article into one of the 5 ...
7,sport,henman hopes ended in dubai third seed tim hen...,Categorize the news article into one of the 5 ...
8,sport,wilkinson fit to face edinburgh england captai...,Categorize the news article into one of the 5 ...
9,entertainment,last star wars not for children the sixth an...,Categorize the news article into one of the 5 ...


In [14]:
df1.to_csv('bbc-text.csv', index=False)

In [15]:
# The instruction dataset to use
dataset_name = "bbc-text.csv"

# Load dataset
dataset = load_dataset("csv", data_files = dataset_name, split = "train")

print(f'Number of prompts: {len(dataset)}')
print(f'Column names are: {dataset.column_names}')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Number of prompts: 2225
Column names are: ['category', 'text', 'Instruction']


In [35]:
dataset1=dataset[0]

In [16]:
print(dataset[randrange(len(dataset))])

{'category': 'sport', 'text': 'chelsea sack mutu chelsea have sacked adrian mutu after he failed a drugs test.  the 25-year-old tested positive for a banned substance - which he later denied was cocaine - in october. chelsea have decided to write off a possible transfer fee for mutu  a £15.8m signing from parma last season  who may face a two-year suspension. a statement from chelsea explaining the decision read: we want to make clear that chelsea has a zero tolerance policy towards drugs.  mutu scored six goals in his first five games after arriving at stamford bridge but his form went into decline and he was frozen out by coach jose mourinho. chelsea s statement added:  this applies to both performance-enhancing drugs or so-called  recreational  drugs. they have no place at our club or in sport.  in coming to a decision on this case  chelsea believed the club s social responsibility to its fans  players  employees and other stakeholders in football regarding drugs was more important 

In [17]:
def create_prompt_formats(sample):
    """
    Creates a formatted prompt template for a prompt in the instruction dataset

    :param sample: Prompt or sample from the instruction dataset
    """

    # Initialize static strings for the prompt template
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruction:"
    INPUT_KEY = "Input:"
    RESPONSE_KEY = "### Response:"
    END_KEY = "### End"

    # Combine a prompt with the static strings
    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{sample['Instruction']}"
    input_context = f"{INPUT_KEY}\n{sample['text']}" if sample["text"] else None
    response = f"{RESPONSE_KEY}\n{sample['category']}"
    end = f"{END_KEY}"

    # Create a list of prompt template elements
    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    # Join prompt template elements into a single string to create the prompt template
    formatted_prompt = "\n\n".join(parts)

    # Store the formatted prompt template in a new key "text"
    sample["Text"] = formatted_prompt

    return sample


In [18]:
print(create_prompt_formats(dataset[randrange(len(dataset))]))

{'category': 'sport', 'text': 'off-colour gardener storms to win britain s jason gardener shook off an upset stomach to win the 60m at sunday s leipzig international meeting.  gardener clocked 6.56 seconds to equal the meeting record and finished well ahead of germany s marc blume  who crossed the line in 6.67 secs. the world indoor champion said:  i got to the airport and my stomach was upset and i was vomiting. i almost went home.  i felt a little better sunday morning but decided i d only run in the main race. then everything went perfectly.  gardener  part of the great britain 4x100m quartet that won gold at the athens olympics  will now turn his attention to next weekend s norwich union european indoor trials in sheffield.   given i am still off-colour i know there is plenty more in the tank and i expect to get faster in the next few weeks   he said.  it s just a case of chipping away as i have done in previous years and the results will come.  scotland s ian mackie was also in ac

In [19]:
def get_max_length(model):
    """
    Extracts maximum token length from the model configuration

    :param model: Hugging Face model
    """

    # Pull model configuration
    conf = model.config
    # Initialize a "max_length" variable to store maximum sequence length as null
    max_length = None
    # Find maximum sequence length in the model configuration and save it in "max_length" if found
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    # Set "max_length" to 1024 (default value) if maximum sequence length is not found in the model configuration
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

In [20]:
def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizes dataset batch

    :param batch: Dataset batch
    :param tokenizer: Model tokenizer
    :param max_length: Maximum number of tokens to emit from the tokenizer
    """

    return tokenizer(
        batch["text"],
        max_length = max_length,
        truncation = True,
    )

In [21]:
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset: str):
    """
    Tokenizes dataset for fine-tuning

    :param tokenizer (AutoTokenizer): Model tokenizer
    :param max_length (int): Maximum number of tokens to emit from the tokenizer
    :param seed: Random seed for reproducibility
    :param dataset (str): Instruction dataset
    """

    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)

    # Apply preprocessing to each batch of the dataset & and remove "instruction", "input", "output", and "text" fields
    _preprocessing_function = partial(preprocess_batch, max_length = max_length, tokenizer = tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched = True,
        remove_columns = ["Instruction", "text", "category", "Text"],
    )

    # Filter out samples that have "input_ids" exceeding "max_length"
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset = dataset.shuffle(seed = seed)

    return dataset

In [22]:
# Random seed
seed = 33

max_length = get_max_length(model)
preprocessed_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset)

print(preprocessed_dataset)



Found max lenth: 4096
Preprocessing dataset...


Map:   0%|          | 0/2225 [00:00<?, ? examples/s]

Map:   0%|          | 0/2225 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2225 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 2222
})


In [23]:
print(preprocessed_dataset[1])

{'input_ids': [1, 10395, 26712, 4287, 1614, 2167, 27550, 4607, 10395, 26712, 4287, 756, 8676, 278, 16920, 310, 967, 4439, 351, 8995, 11020, 411, 263, 716, 27550, 29871, 278, 263, 374, 653, 29889, 29871, 515, 7398, 388, 29871, 599, 26094, 322, 8078, 29879, 674, 505, 304, 367, 23153, 297, 278, 263, 374, 653, 29871, 607, 471, 3534, 292, 472, 29871, 29896, 29871, 29947, 29929, 29941, 304, 278, 502, 11232, 279, 29889, 278, 4439, 351, 8995, 11020, 29871, 607, 5714, 4359, 4203, 967, 995, 297, 29871, 29906, 29900, 29900, 29946, 29871, 338, 694, 5520, 11706, 22707, 541, 674, 3933, 14523, 519, 472, 24388, 2745, 29871, 29906, 29900, 29900, 29929, 29889, 278, 1374, 5832, 714, 310, 278, 11020, 29871, 23580, 297, 5757, 29891, 29871, 29906, 29900, 29900, 29941, 29871, 471, 9146, 304, 5418, 278, 4234, 515, 967, 4940, 1090, 285, 4615, 25539, 5751, 322, 3211, 278, 1108, 310, 278, 2919, 5253, 310, 6795, 1725, 277, 2524, 2395, 297, 18342, 362, 29889, 29871, 372, 269, 2038, 599, 263, 1139, 310, 577, 9359, 

In [24]:
def create_peft_config(r, lora_alpha, target_modules, lora_dropout, bias, task_type):
    """
    Creates Parameter-Efficient Fine-Tuning configuration for the model

    :param r: LoRA attention dimension
    :param lora_alpha: Alpha parameter for LoRA scaling
    :param modules: Names of the modules to apply LoRA to
    :param lora_dropout: Dropout Probability for LoRA layers
    :param bias: Specifies if the bias parameters should be trained
    """
    config = LoraConfig(
        r = r,
        lora_alpha = lora_alpha,
        target_modules = target_modules,
        lora_dropout = lora_dropout,
        bias = bias,
        task_type = task_type,
    )

    return config

In [25]:
def find_all_linear_names(model):
    """
    Find modules to apply LoRA to.

    :param model: PEFT model
    """

    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    print(f"LoRA module names: {list(lora_module_names)}")
    return list(lora_module_names)

In [26]:
def print_trainable_parameters(model, use_4bit = False):
    """
    Prints the number of trainable parameters in the model.

    :param model: PEFT model
    """

    trainable_params = 0
    all_param = 0

    for _, param in model.named_parameters():
        num_params = param.numel()
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel
        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params

    if use_4bit:
        trainable_params /= 2

    print(
        f"All Parameters: {all_param:,d} || Trainable Parameters: {trainable_params:,d} || Trainable Parameters %: {100 * trainable_params / all_param}"
    )

In [27]:
def fine_tune(model,
          tokenizer,
          dataset,
          lora_r,
          lora_alpha,
          lora_dropout,
          bias,
          task_type,
          per_device_train_batch_size,
          gradient_accumulation_steps,
          warmup_steps,
          max_steps,
          learning_rate,
          fp16,
          logging_steps,
          output_dir,
          optim):
    """
    Prepares and fine-tune the pre-trained model.

    :param model: Pre-trained Hugging Face model
    :param tokenizer: Model tokenizer
    :param dataset: Preprocessed training dataset
    """

    # Enable gradient checkpointing to reduce memory usage during fine-tuning
    model.gradient_checkpointing_enable()

    # Prepare the model for training
    model = prepare_model_for_kbit_training(model)

    # Get LoRA module names
    target_modules = find_all_linear_names(model)

    # Create PEFT configuration for these modules and wrap the model to PEFT
    peft_config = create_peft_config(lora_r, lora_alpha, target_modules, lora_dropout, bias, task_type)
    model = get_peft_model(model, peft_config)

    # Print information about the percentage of trainable parameters
    print_trainable_parameters(model)

    # Training parameters
    trainer = Trainer(
        model = model,
        train_dataset = dataset,
        args = TrainingArguments(
            per_device_train_batch_size = per_device_train_batch_size,
            gradient_accumulation_steps = gradient_accumulation_steps,
            warmup_steps = warmup_steps,
            max_steps = max_steps,
            learning_rate = learning_rate,
            fp16 = fp16,
            logging_steps = logging_steps,
            output_dir = output_dir,
            optim = optim,
        ),
        data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False)
    )

    model.config.use_cache = False

    do_train = True

    # Launch training and log metrics
    print("Training...")

    if do_train:
        train_result = trainer.train()
        metrics = train_result.metrics
        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()
        print(metrics)

    # Save model
    print("Saving last checkpoint of the model...")
    os.makedirs(output_dir, exist_ok = True)
    trainer.model.save_pretrained(output_dir)

    # Free memory for merging weights
    del model
    del trainer
    torch.cuda.empty_cache()

In [28]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 16

# Alpha parameter for LoRA scaling
lora_alpha = 64

# Dropout probability for LoRA layers
lora_dropout = 0.1

# Bias
bias = "none"

# Task type
task_type = "CAUSAL_LM"

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Batch size per GPU for training
per_device_train_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 4

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Optimizer to use
optim = "paged_adamw_32bit"

# Number of training steps (overrides num_train_epochs)
max_steps = 20

# Linear warmup steps from 0 to learning_rate
warmup_steps = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True

# Log every X updates steps
logging_steps = 1

In [29]:
fine_tune(model, tokenizer, preprocessed_dataset, lora_r, lora_alpha, lora_dropout, bias, task_type, per_device_train_batch_size, gradient_accumulation_steps, warmup_steps, max_steps, learning_rate, fp16, logging_steps, output_dir, optim)

LoRA module names: ['up_proj', 'o_proj', 'v_proj', 'q_proj', 'down_proj', 'gate_proj', 'k_proj']
All Parameters: 3,540,389,888 || Trainable Parameters: 39,976,960 || Trainable Parameters %: 1.1291682911958425
Training...


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,2.061600
2,2.121800
3,2.292300
4,2.061200
5,2.068300
6,2.005200
7,1.978700
8,1.921800
9,2.081800
10,1.833900


***** train metrics *****
  epoch                    =       0.04
  total_flos               =   931710GF
  train_loss               =     2.0112
  train_runtime            = 0:04:59.38
  train_samples_per_second =      0.267
  train_steps_per_second   =      0.067
{'train_runtime': 299.3836, 'train_samples_per_second': 0.267, 'train_steps_per_second': 0.067, 'total_flos': 1000416603783168.0, 'train_loss': 2.0111522912979125, 'epoch': 0.04}
Saving last checkpoint of the model...


In [30]:
def summarize(model, text: str):
    inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.0001)
    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)

In [37]:
# Random seed
seed = 33

max_length = get_max_length(model)
test_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset1)

print(test_dataset[0])

Found max lenth: 4096
Preprocessing dataset...


AttributeError: ignored

In [38]:
test_token=tokenizer(
        "pakistan facing alot",
        max_length = max_length,
        truncation = True)

In [39]:
test_token

{'input_ids': [1, 27068, 9777, 14870, 394, 327], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [43]:
summarize(model,test_token.tolist())

AttributeError: ignored

In [32]:
type(model)

transformers.models.llama.modeling_llama.LlamaForCausalLM

In [50]:
from google.colab import files
import shutil

shutil.make_archive("results", 'zip', "results")
files.download("results.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:
model.generate(test_token,max_new_tokens=256, temperature=0.0001)

AttributeError: ignored

In [49]:
text_to_generate = "Categorize the following text into one of the 5 categories:\n\ntech\nbusiness\nsport\nentertainment\npolitics. Pakistan is facing alot"

# Tokenize the input text
input_ids = tokenizer.encode(text_to_generate, return_tensors="pt")
input_ids = input_ids.to('cuda')
# Generate output from the model
output = model.generate(input_ids,max_new_tokens=40)

# Decode the generated output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:", generated_text)

Generated Text: Categorize the following text into one of the 5 categories:

tech
business
sport
entertainment
politics. Pakistan is facing alot of problems. The country is facing a severe energy crisis. The country is facing a severe energy crisis. The country is facing a severe energy crisis. The country is facing a severe energy crisis. The


In [53]:

from transformers import LocalAgent

agent = LocalAgent(model,tokenizer)
agent.run("Categorize the following text into one of the 5 categories:\n\ntech\nbusiness\nsport\nentertainment\npolitics", text="Pakistan is facing issues")

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


==Explanation from the agent==
I will use the following tool: `text_classifier` to classify the text.


==Code generated by the agent==
answer = text_classifier(text=text, labels=["tech", "business", "sport", "entertainment", "politics"])
print(f"The most likely label is {answer}.")


==Result==


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

The most likely label is politics.


'politics'